In [8]:
import pandas as pd
import jsonlines
import ast

from pprint import pprint

In [10]:
data = pd.read_csv('data/medicine_data_processed.csv')
data.head()

,Unnamed: 0,sub_category,product_name,salt_composition,product_manufactured,medicine_desc,side_effects,drug,brand,effect
0,0,Human Insulin Basal,Human Insulatard 40IU/ml Suspension for Injection,Insulin Isophane (40IU),Novo Nordisk India Pvt Ltd,Human Insulatard 40IU/ml Suspension for Inject...,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","[' Apriace', ' Capotril, Aceten, Angiopril', '...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
1,1,Human Insulin Basal,Insulin 40IU/ml Injection,Insulin Isophane (40IU),Sun Pharmaceutical Industries Ltd,Insulin 40IU/ml Injection is used to improve b...,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","[' Apriace', ' Capotril, Aceten, Angiopril', '...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
2,2,Human Insulin Basal,Huminsulin N 40IU/ml Injection,Insulin Isophane (40IU),Eli Lilly and Company India Pvt Ltd,Huminsulin N 40IU/ml Injection is used to impr...,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","[' Apriace', ' Capotril, Aceten, Angiopril', '...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
3,3,Human Insulin Basal,Insugen-N 40IU/ml Injection,Insulin Isophane (40IU),Biocon,Insugen-N 40IU/ml Injection is used to improve...,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","[' Apriace', ' Capotril, Aceten, Angiopril', '...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
4,4,Human Insulin Basal,Insulatard 100IU/ml Flexpen,Insulin Isophane (100IU/ml),Novo Nordisk India Pvt Ltd,Insulatard 100IU/ml Flexpen is used to improve...,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","[' Apriace', ' Capotril, Aceten, Angiopril', '...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"


In [39]:
prompt_template = """Drug1: {drug1}
Drug2: {drug2}

Side effects are {side_effects}

Drug Ineraction effect can be {effect}
"""

In [11]:
data = data[['product_name', 'side_effects', 'drug', 'effect']]
data.head()

,product_name,side_effects,drug,effect
0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
1,Insulin 40IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
2,Huminsulin N 40IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
3,Insugen-N 40IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"
4,Insulatard 100IU/ml Flexpen,"Hypoglycemia (low blood glucose level),Injecti...","['Benazepril', 'Captopril', 'Enalapril', 'Fosi...","['MODERATE', 'MODERATE', 'MODERATE', 'MODERATE']"


In [16]:
data['drug'] = data['drug'].apply(lambda x: ast.literal_eval(x))
data['effect'] = data['effect'].apply(lambda x: ast.literal_eval(x))

In [17]:
exploded_data = data.explode(['drug', 'effect'])
exploded_data.head()

,product_name,side_effects,drug,effect
0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Benazepril,MODERATE
0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Captopril,MODERATE
0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Enalapril,MODERATE
0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Fosinopril,MODERATE
1,Insulin 40IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...",Benazepril,MODERATE


In [28]:
exploded_data.reset_index(inplace=True)

In [29]:
exploded_data[exploded_data.duplicated(subset=['product_name', 'side_effects', 'drug', 'effect'])]

,index,product_name,side_effects,drug,effect
88,22,Lupisulin N 100IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...",Benazepril,MODERATE
89,22,Lupisulin N 100IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...",Captopril,MODERATE
90,22,Lupisulin N 100IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...",Enalapril,MODERATE
91,22,Lupisulin N 100IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...",Fosinopril,MODERATE
272,68,Novorapid 100IU/ml Solution for Injection,"Hypoglycemia (low blood glucose level),Lipodys...",Amiloride,SERIOUS
...,...,...,...,...,...
286957,77529,V D3 Sachet,No common side effects seen,Orlistat,SERIOUS
286958,77530,Alpha D3 0.5mcg Capsule,"Rash,Itching,Abdominal pain,Increased calcium ...",Orlistat,SERIOUS
286959,77531,Raisecal D3 60K Softgel Capsule,No common side effects seen,Orlistat,SERIOUS
286988,77546,K-Pot 150mg Injection,"Injection site reactions (pain, swelling, redn...",Leflunomide,LIFE-THREATENING


In [40]:
sample_data = exploded_data.loc[0]

sample_output = prompt_template.format(
    drug1 = sample_data['product_name'],
    drug2 = sample_data['drug'],
    side_effects = sample_data['side_effects'],
    effect = sample_data['effect']
)
print(sample_output)

Drug1: Human Insulatard 40IU/ml Suspension for Injection
Drug2: Benazepril

Side effects are Hypoglycemia (low blood glucose level),Injection site allergic reaction,Lipodystrophy (skin thickening or pits at the injection site),Weight gain,Edema (swelling)

Drug Ineraction effect can be MODERATE



In [45]:
def generate_response(row):
    drug1 = row['product_name']
    side_effects = row['side_effects']
    drug2 = row['drug']
    effect = row['effect']

    return prompt_template.format(
        drug1 = drug1,
        drug2 = drug2,
        side_effects = side_effects,
        effect = effect
    )

exploded_data['output'] = exploded_data.apply(generate_response, axis=1)

In [46]:
exploded_data

,index,product_name,side_effects,drug,effect,output
0,0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Benazepril,MODERATE,Drug1: Human Insulatard 40IU/ml Suspension for...
1,0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Captopril,MODERATE,Drug1: Human Insulatard 40IU/ml Suspension for...
2,0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Enalapril,MODERATE,Drug1: Human Insulatard 40IU/ml Suspension for...
3,0,Human Insulatard 40IU/ml Suspension for Injection,"Hypoglycemia (low blood glucose level),Injecti...",Fosinopril,MODERATE,Drug1: Human Insulatard 40IU/ml Suspension for...
4,1,Insulin 40IU/ml Injection,"Hypoglycemia (low blood glucose level),Injecti...",Benazepril,MODERATE,Drug1: Insulin 40IU/ml Injection\nDrug2: Benaz...
...,...,...,...,...,...,...
286985,77544,Ricetral 0.3gm Injection,"Injection site reactions (pain, swelling, redn...",Penicillamine,LIFE-THREATENING,Drug1: Ricetral 0.3gm Injection\nDrug2: Penici...
286986,77545,Tuberium 150mg Injection,"Injection site reactions (pain, swelling, redn...",Leflunomide,LIFE-THREATENING,Drug1: Tuberium 150mg Injection\nDrug2: Leflun...
286987,77545,Tuberium 150mg Injection,"Injection site reactions (pain, swelling, redn...",Penicillamine,LIFE-THREATENING,Drug1: Tuberium 150mg Injection\nDrug2: Penici...
286988,77546,K-Pot 150mg Injection,"Injection site reactions (pain, swelling, redn...",Leflunomide,LIFE-THREATENING,Drug1: K-Pot 150mg Injection\nDrug2: Leflunomi...


In [50]:
num_examples = len(exploded_data['product_name'])
finetuning_dataset = []
for i in range(num_examples):
    input1 = exploded_data['product_name'][i]
    input2 = exploded_data['drug'][i]
    answer = exploded_data['output'][i]
    finetuning_dataset.append({'input1':input1, 'input2':input2, 'output':answer})

In [53]:
pprint(finetuning_dataset[0])

{'input1': 'Human Insulatard 40IU/ml Suspension for Injection',
 'input2': 'Benazepril',
 'output': 'Drug1: Human Insulatard 40IU/ml Suspension for Injection\n'
           'Drug2: Benazepril\n'
           '\n'
           'Side effects are Hypoglycemia (low blood glucose level),Injection '
           'site allergic reaction,Lipodystrophy (skin thickening or pits at '
           'the injection site),Weight gain,Edema (swelling)\n'
           '\n'
           'Drug Ineraction effect can be MODERATE\n'}


In [54]:
import jsonlines
with jsonlines.open(f'data/drug_interaction.jsonl', 'w') as writer:
    writer.write_all(finetuning_dataset)

In [3]:
import json

def split_jsonl(file_path, output_file1, output_file2):
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()
    
    total_lines = len(lines)
    half_point = total_lines // 5
    
    # Split the lines into two halves
    lines_first_half = lines[:half_point]
    lines_second_half = lines[half_point:]
    
    # Write the first half to output_file1
    with open(output_file1, 'w', encoding='utf-8') as f1:
        for line in lines_first_half:
            f1.write(line)
    
    # Write the second half to output_file2
    with open(output_file2, 'w', encoding='utf-8') as f2:
        for line in lines_second_half:
            f2.write(line)

# Example usage
file_path = 'data/drug_interaction.jsonl'
output_file1 = 'data/drug_interaction1.jsonl'
output_file2 = 'data/drug_interaction2.jsonl'
split_jsonl(file_path, output_file1, output_file2)
